In [18]:
!pip3 install indic-nlp-library

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
import torch

In [20]:
from indicnlp.tokenize import indic_tokenize

In [21]:
data = pd.read_csv(
    "/kaggle/input/language-translation1/Sentence pairs in English-Hindi - 2025-02-11.tsv",
    sep="\t",
    header=None,
    names=["SrcSennId","SrcSent","DatSentID","DatSent"]
)

In [22]:
data.head()

,SrcSennId,SrcSent,DatSentID,DatSent
0,1282,Muiriel is 20 now.,485968,म्यूरियल अब बीस साल की हो गई है।
1,1282,Muiriel is 20 now.,2060319,म्यूरियल अब बीस साल की है।
2,1294,Education in this world disappoints me.,485564,मैं इस दुनिया में शिक्षा पर बहुत निराश हूँ।
3,1302,That won't happen.,2060320,वैसा नहीं होगा।
4,1308,I miss you.,2060321,मुझें तुम्हारी याद आ रही है।


In [23]:
# data.drop(labels=[data.columns[0], data.columns[2]], axis=1, inplace=True)
# Save column names first
cols_to_drop = [data.columns[0], data.columns[2]]

# Drop
data.drop(columns=cols_to_drop, inplace=True)

In [24]:
from transformers import AutoTokenizer

src_sent_tokenizer = AutoTokenizer.from_pretrained("t5-base")


In [25]:
data["SrcSent"] = data["SrcSent"].apply(lambda x: src_sent_tokenizer.tokenize(x))


In [26]:
data.head()

,SrcSent,DatSent
0,"[▁Mu, i, riel, ▁is, ▁20, ▁now, .]",म्यूरियल अब बीस साल की हो गई है।
1,"[▁Mu, i, riel, ▁is, ▁20, ▁now, .]",म्यूरियल अब बीस साल की है।
2,"[▁Education, ▁in, ▁this, ▁world, ▁disappoint, ...",मैं इस दुनिया में शिक्षा पर बहुत निराश हूँ।
3,"[▁That, ▁won, ', t, ▁happen, .]",वैसा नहीं होगा।
4,"[▁I, ▁miss, ▁you, .]",मुझें तुम्हारी याद आ रही है।


In [27]:
# # data["DstSent"]= data["DstSent"].apply(lambda x: indic_tokenize.trivial_tokenize(x,lang="hi")
# data["DstSent"] = data["DstSent"].apply(
#     lambda x: indic_tokenize.trivial_tokenize(x, lang="hi")
# )
from indicnlp.tokenize import indic_tokenize

data["DatSent"] = data["DatSent"].apply(
    lambda x: indic_tokenize.trivial_tokenize(x, lang="hi")
)


In [28]:
print(data.columns)


Index(['SrcSent', 'DatSent'], dtype='object')


In [29]:
# data["SrcSent"]= data["srcSent"].apply(arc_sent_tokenizer.convert_tokens_to_ids)
data["SrcSent_ids"] = data["SrcSent"].apply(
    lambda tokens: src_sent_tokenizer.convert_tokens_to_ids(tokens)
)



In [30]:
Vs = src_sent_tokenizer.get_vocab()


In [36]:
# Vd = set()
# for tokenized_hindi_sent in data["DatSent"]:
#     hindi_vocab.update(tokenized_hindi_sent)

Vd = set()
for sent in data["DatSent"]:
    # Convert to string (avoid NaN, lists, etc.)
    sent = str(sent)
    tokens = tokenizer.tokenize(sent)
    Vd.update(tokens)

print("Unique tokens in dataset:", len(Vd))
print(list(Vd)[:20])



Unique tokens in dataset: 557
['रण', 'दखल', 'टन', 'लगभग', 'रप', 'थन', '२२', 'रभ', 'नम', 'भर', 'नयन', '१९', 'ञ', 'यर', 'चढ', 'यश', 'i', 'तम', 'इत', 'ऊ']


In [37]:
hindi_vocab = set()

for tokenized_hindi_sent in data["DatSent"]:
    hindi_vocab.update(tokenized_hindi_sent)


In [38]:
Vd = dict()

# Special tokens
Vd["<PAD>"] = 0
Vd["<SOS>"] = 1
Vd["<EOS>"] = 2

# Add rest of vocab
for idx, token in enumerate(hindi_vocab, start=3):
    Vd[token] = idx


In [39]:
def convert_hindi_token_to_ids(hindi_sent):
    return[Vd[token] for token in hindi_sent]
    

In [40]:
data["DatSent"] = data["DatSent"].apply(
    lambda x: convert_hindi_token_to_ids(x)
)


In [41]:
def insert_sos_token_id(hindi_sent_token_ids):
    return [1] + hindi_sent_token_ids


In [42]:
data["DatSentInput"] = data["DatSent"].apply(lambda x: insert_sos_token_id(x))


In [43]:
data.head()

,SrcSent,DatSent,SrcSent_ids,DatSentInput
0,"[▁Mu, i, riel, ▁is, ▁20, ▁now, .]","[81, 2013, 1229, 1141, 1223, 5561, 6506, 1837,...","[4159, 23, 14018, 19, 460, 230, 5]","[1, 81, 2013, 1229, 1141, 1223, 5561, 6506, 18..."
1,"[▁Mu, i, riel, ▁is, ▁20, ▁now, .]","[81, 2013, 1229, 1141, 1223, 1837, 3892]","[4159, 23, 14018, 19, 460, 230, 5]","[1, 81, 2013, 1229, 1141, 1223, 1837, 3892]"
2,"[▁Education, ▁in, ▁this, ▁world, ▁disappoint, ...","[2098, 583, 3740, 581, 2222, 1873, 4710, 3956,...","[2855, 16, 48, 296, 26963, 7, 140, 5]","[1, 2098, 583, 3740, 581, 2222, 1873, 4710, 39..."
3,"[▁That, ▁won, ', t, ▁happen, .]","[1527, 3668, 3811, 3892]","[466, 751, 31, 17, 1837, 5]","[1, 1527, 3668, 3811, 3892]"
4,"[▁I, ▁miss, ▁you, .]","[6158, 1058, 1210, 6767, 4096, 1837, 3892]","[27, 3041, 25, 5]","[1, 6158, 1058, 1210, 6767, 4096, 1837, 3892]"


In [44]:
def insert_eos_token_id(hindi_sent_token_ids):
    return hindi_sent_token_ids + [2]


In [45]:
data["DatSentLavel"] = data["DatSent"].apply(
    lambda x: insert_eos_token_id(x)
)


In [46]:
data.head()


,SrcSent,DatSent,SrcSent_ids,DatSentInput,DatSentLavel
0,"[▁Mu, i, riel, ▁is, ▁20, ▁now, .]","[81, 2013, 1229, 1141, 1223, 5561, 6506, 1837,...","[4159, 23, 14018, 19, 460, 230, 5]","[1, 81, 2013, 1229, 1141, 1223, 5561, 6506, 18...","[81, 2013, 1229, 1141, 1223, 5561, 6506, 1837,..."
1,"[▁Mu, i, riel, ▁is, ▁20, ▁now, .]","[81, 2013, 1229, 1141, 1223, 1837, 3892]","[4159, 23, 14018, 19, 460, 230, 5]","[1, 81, 2013, 1229, 1141, 1223, 1837, 3892]","[81, 2013, 1229, 1141, 1223, 1837, 3892, 2]"
2,"[▁Education, ▁in, ▁this, ▁world, ▁disappoint, ...","[2098, 583, 3740, 581, 2222, 1873, 4710, 3956,...","[2855, 16, 48, 296, 26963, 7, 140, 5]","[1, 2098, 583, 3740, 581, 2222, 1873, 4710, 39...","[2098, 583, 3740, 581, 2222, 1873, 4710, 3956,..."
3,"[▁That, ▁won, ', t, ▁happen, .]","[1527, 3668, 3811, 3892]","[466, 751, 31, 17, 1837, 5]","[1, 1527, 3668, 3811, 3892]","[1527, 3668, 3811, 3892, 2]"
4,"[▁I, ▁miss, ▁you, .]","[6158, 1058, 1210, 6767, 4096, 1837, 3892]","[27, 3041, 25, 5]","[1, 6158, 1058, 1210, 6767, 4096, 1837, 3892]","[6158, 1058, 1210, 6767, 4096, 1837, 3892, 2]"


In [47]:
data.drop(labels=[data.columns[1]], axis=1, inplace=True)


In [48]:
print(data.columns)


Index(['SrcSent', 'SrcSent_ids', 'DatSentInput', 'DatSentLavel'], dtype='object')


In [49]:
X = list(data["SrcSent"])
Y_input = list(data["DatSentInput"])
Y_label = list(data["DatSentLavel"])


In [50]:
X_tensor = [torch.tensor(ids) for ids in data["SrcSent_ids"]]
Y_input_tensor = [torch.tensor(ids) for ids in data["DatSentInput"]]  # if these are already token IDs
Y_label_tensor = [torch.tensor(ids) for ids in data["DatSentLavel"]]   # if these are already token IDs


In [51]:
X_padded = torch.nn.utils.rnn.pad_sequence(X_tensor,batch_first=True)
Y_padded_input = torch.nn.utils.rnn.pad_sequence(Y_input_tensor,batch_first=True)
Y_padded_label = torch.nn.utils.rnn.pad_sequence(Y_label_tensor,batch_first=True)

In [52]:

Ns = X_padded.shape[1]
Nd = Y_padded_label.shape[1]

In [54]:
# class Encoder(torch.nn.module):
#     def __init__(self.src,src_lang_vocab_size,word_embedding_dim):
#         super(Encoder,self),__init__()
#         self.first_embedding_layer = torch.nn.embedding(num.embedding=src_lang_VoCsL_sIZE,Embedding.dim)
#         self.second_lstm_layer = torch.nn.LSTM() input_size = word_embedding_dim,
#         hidden_size=word_embedding_dim,
#         batch first= True:
        
        

In [56]:
import torch
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, src_lang_vocab_size, word_embedding_dim, hidden_size, num_layers=1):
        super(Encoder, self).__init__()
        
        # Embedding layer
        self.embedding = nn.Embedding(num_embeddings=src_lang_vocab_size,
                                      embedding_dim=word_embedding_dim)
        
        # LSTM layer
        self.lstm = nn.LSTM(input_size=word_embedding_dim,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            batch_first=True)
    
    def forward(self, src):
        # src shape: (batch_size, seq_len)
        embedded = self.embedding(src)   # (batch_size, seq_len, word_embedding_dim)
        outputs, (hidden, cell) = self.lstm(embedded)
        return outputs, (hidden, cell)
